# Load dataset from kaggle

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "nadaazabmohamed" # from kaggle.json
os.environ['KAGGLE_KEY'] =  "" # from kaggle.json

In [ ]:
!kaggle competitions download -c support23-fashion-classification

support23-fashion-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/support23-fashion-classification.zip

Archive:  /content/support23-fashion-classification.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
import csv
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier,BaggingClassifier,GradientBoostingClassifier,StackingClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss,accuracy_score

Mount drive folder to workspace

# Data loading

In [ ]:
datatrain=pd.read_csv("train.csv")
datatest=pd.read_csv("test.csv")

In [ ]:
datatrain.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,0,0,0,0,0,0,14,85,76,...,45,52,102,2,0,2,0,0,0,0
1,1,0,0,0,0,0,0,0,0,34,...,193,49,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3,0,0,0,0,0,2,0,0,47,...,144,97,67,22,0,1,0,0,0,0


In [ ]:
datatest.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,88,14,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,30,86,0,...,15,38,77,73,0,0,2,0,0,0
2,0,0,0,0,1,0,0,0,0,48,...,6,17,0,0,9,193,208,113,0,0
3,0,0,0,0,0,0,0,5,97,166,...,233,226,215,234,31,0,2,0,0,0
4,0,0,0,0,0,0,0,0,0,4,...,0,0,0,2,0,2,50,0,0,0


# Train a Decision Tree and a Random Forest model using [scikit-learn](https://scikit-learn.org/stable/)

In [ ]:
# X is the variable holding your samples
# y is the variable holding your labels
X=datatrain.iloc[:,1:785].values
y=datatrain['label']
X.shape

(40000, 784)

In [ ]:
y

0        3
1        1
2        3
3        8
4        3
        ..
39995    2
39996    1
39997    4
39998    3
39999    1
Name: label, Length: 40000, dtype: int64

In [ ]:
# data label 
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)
y_enc = one_hot_encoder.fit_transform(datatrain.iloc[:,0:1])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
y_enc

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [ ]:
#split dataset to train and test
X_train,X_test,y_train,y_test=train_test_split(X,y_enc,test_size=.2,random_state=2)
y_train

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
#split dataset to train and test
X_train_not,X_test_not,y_train_not,y_test_not=train_test_split(X,y,test_size=.2,random_state=0)
y_train_not

21370    9
2470     4
13767    4
13316    8
26374    7
        ..
20757    1
32103    4
30403    8
21243    7
2732     6
Name: label, Length: 32000, dtype: int64

# ***DecisionTree***

In [ ]:
DecisionTree=DecisionTreeClassifier(max_depth=2)
DecisionTree.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=2)

In [ ]:
y_pred=DecisionTree.predict(X_test)

In [ ]:
accuracy=accuracy_score(y_test,y_pred)
accuracy

0.1745

In [ ]:
# save your model
pickle.dump(DecisionTree,open("DecisionTree.pkl",'wb'))

# **RandomForest**

In [ ]:
RandomForest=RandomForestClassifier(max_leaf_nodes=85, max_depth=85)
RandomForest.fit(X_train,y_train)

RandomForestClassifier(max_depth=85, max_leaf_nodes=85)

In [ ]:
y_pred_Random=RandomForest.predict(X_test)

In [ ]:
accuaracy_Random=accuracy_score(y_test,y_pred_Random)
accuaracy_Random

0.716875

In [ ]:
pickle.dump(RandomForest,open("RandomForest.pkl",'wb'))

# ***logistic***

In [ ]:
from sklearn.linear_model import LogisticRegression
logistic=LogisticRegression(multi_class='ovr',max_iter=100,)
logistic.fit(X_train_not,y_train_not)
y_pred_model=logistic.predict(X_test_not)
accurecy_sk=accuracy_score(y_test_not,y_pred_model)
accurecy_sk

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8385

In [ ]:
pickle.dump(logistic,open("logistic.pkl",'wb'))

# ***AdaBoost***

In [ ]:
# Initialize AdaBoost with the base learner
adaboost = AdaBoostClassifier(base_estimator=DecisionTree, n_estimators=1000, learning_rate=.1, random_state=42)
adaboost.fit(X_train_not,y_train_not)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2),
                   learning_rate=0.1, n_estimators=1000, random_state=42)

In [ ]:
y_pred_ada=adaboost.predict(X_test_not)

In [ ]:
from sklearn.metrics import f1_score
accuaracy_ada=f1_score(y_test_not,y_pred_ada)
accuaracy_ada

ValueError: ignored

In [ ]:
acc=accuracy_score(y_test_not,y_pred_ada)
acc

0.61225

In [ ]:
pickle.dump(AdaBoostClassifier,open("AdaBoostClassifier.pkl",'wb'))

# ***Bagging***

In [ ]:
from sklearn.model_selection import KFold
seed=7
kfold=KFold(n_splits=10,random_state=seed,shuffle=True)

In [ ]:
BaggingClassifier=BaggingClassifier(base_estimator=DecisionTree, n_estimators=10,random_state=seed,n_jobs=5)


In [ ]:
from sklearn.model_selection import cross_val_score
result=cross_val_score(BaggingClassifier,X_train_not,y_train_not,cv=kfold)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/en

In [ ]:
BaggingClassifier.fit(X_train_not,y_train_not)
y_pred_bag=BaggingClassifier.predict(X_test_not)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [ ]:
from sklearn.metrics import f1_score
accuaracy_bag=f1_score(y_test_not,y_pred_bag,average='weighted')
accuaracy_bag

0.8456772876728207

In [ ]:
pickle.dump(BaggingClassifier,open("BaggingClassifier.pkl",'wb'))

# ***GradientBoosting***

In [ ]:
GradientBoostingClassifier=GradientBoostingClassifier(n_estimators=20, learning_rate=1,max_depth=1, random_state=0)
GradientBoostingClassifier.fit(X_train_not,y_train_not)

GradientBoostingClassifier(learning_rate=1, max_depth=1, n_estimators=20,
                           random_state=0)

In [ ]:
y_pred_grad=GradientBoostingClassifier.predict(X_test_not)

In [ ]:
accuaracy_grad=f1_score(y_test_not,y_pred_grad,average='weighted')
accuaracy_grad

0.6799424412711572

In [ ]:
pickle.dump(GradientBoostingClassifier,open("GradientBoostingClassifier.pkl",'wb'))

# ***Stacking***

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import StackingClassifier
clf = [('dec',DecisionTree),('bag',BaggingClassifier)]
#estimators=[('log',LogisticRegression(multi_class='ovr')),('bag',BaggingClassifier(base_estimator=DecisionTree, n_estimators=80,random_state=4,n_jobs=5))]
StackingClassifie=StackingClassifier(estimators=clf,final_estimator=RandomForest , )
score = cross_val_score(StackingClassifie,X_train_not,y_train_not,cv = 3,scoring = 'accuracy',error_score='raise')
StackingClassifie.fit(X_train_not,y_train_not)

TypeError: ignored

In [ ]:
xgb_model = XGBClassifier(objective="multi:softprob", random_state=5, n_estimators=700)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import StackingClassifier
dtc =  DecisionTreeClassifier()
rfc = RandomForestClassifier()
xgb = XGBClassifier()
clf = [('dtc',dtc),('rfc',rfc),('xgb',xgb)]
lr = LogisticRegression()
stack_model = StackingClassifier( estimators = clf,final_estimator = lr)
score = cross_val_score(stack_model,X_train_not,y_train_not,cv = 5,scoring = 'accuracy')

NameError: ignored

In [ ]:
stack_model.fit(X_train_not,y_train_not)
y_pred_stack=stack_model.predict(X_test_not)
accuaracy_stack=f1_score(y_test_not,y_pred_stack,average='weighted')
accuaracy_stack

NameError: ignored

In [ ]:
pickle.dump(StackingClassifier,open("StackingClassifier.pkl",'wb'))

# ***CatBoost***

In [ ]:
CatBoostClassifier=CatBoostClassifier(iterations=10000,
                           learning_rate=1,
                           depth=3)
CatBoostClassifier.fit(X_train_not,y_train_not)

Streaming output truncated to the last 5000 lines.
5000:	learn: 0.0048319	total: 1h 5m 2s	remaining: 1h 5m
5001:	learn: 0.0048316	total: 1h 5m 3s	remaining: 1h 4m 59s
5002:	learn: 0.0048288	total: 1h 5m 3s	remaining: 1h 4m 59s
5003:	learn: 0.0048282	total: 1h 5m 4s	remaining: 1h 4m 58s
5004:	learn: 0.0048229	total: 1h 5m 5s	remaining: 1h 4m 58s
5005:	learn: 0.0048225	total: 1h 5m 7s	remaining: 1h 4m 57s
5006:	learn: 0.0048200	total: 1h 5m 7s	remaining: 1h 4m 56s
5007:	learn: 0.0048171	total: 1h 5m 8s	remaining: 1h 4m 55s
5008:	learn: 0.0048156	total: 1h 5m 9s	remaining: 1h 4m 55s
5009:	learn: 0.0048121	total: 1h 5m 9s	remaining: 1h 4m 54s
5010:	learn: 0.0048089	total: 1h 5m 10s	remaining: 1h 4m 53s
5011:	learn: 0.0048018	total: 1h 5m 11s	remaining: 1h 4m 52s
5012:	learn: 0.0048005	total: 1h 5m 12s	remaining: 1h 4m 51s
5013:	learn: 0.0047980	total: 1h 5m 12s	remaining: 1h 4m 50s
5014:	learn: 0.0047952	total: 1h 5m 13s	remaining: 1h 4m 50s
5015:	learn: 0.0047922	total: 1h 5m 14s	remainin

In [ ]:
y_pred_cat=CatBoostClassifier.predict(X_test_not)

In [ ]:
accuaracy_cat=accuracy_score(y_test_not,y_pred_cat)
accuaracy_cat

0.8825

In [ ]:
score=f1_score(y_test_not,y_pred_cat,average='weighted')
score

0.8633263575457008

In [ ]:
pickle.dump(CatBoostClassifier,open("CatBoostClassifier.pkl",'wb'))

# ***XGB***

In [ ]:
xgb_model = XGBClassifier(objective="multi:softprob", random_state=5, n_estimators=700)
xgb_model.fit(X_train_not,y_train_not)#1.5h for this accurcy

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=700, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [ ]:
y_pred_XG=xgb_model.predict(X_test_not)

In [ ]:
accuaracy_XG=accuracy_score(y_test_not,y_pred_XG)
accuaracy_XG

0.902875

In [ ]:
score=f1_score(y_test_not,y_pred_XG,average='weighted')
score

0.9024730494913434

In [ ]:
pickle.dump(xgb_model,open("XG.pkl",'wb'))

the faster with v.good accurcy :CatBoost -->XG,,,,
the slowest  with v.good accurcy:stack,,

In [ ]:
y_predtest_XG=xgb_model.predict(datatest)

In [ ]:
submission_id=pd.DataFrame(np.arange(0,10000,1))
submission_label=pd.DataFrame(data=y_predtest_XG).astype('int')
submission_id.insert(1, column = "Label", value =submission_label)  
#submission_id.rename(columns={"0": "Id"}, inplace=True)
submission_id.columns = ["Id", "Label"]
submission_id.to_csv('submission.csv', index=False)

In [ ]:
submission_id.head()

,Id,Label
0,0,3
1,1,6
2,2,2
3,3,0
4,4,9
